In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/bayes

/content/drive/MyDrive/bayes


In [5]:
import nltk

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

<div dir="rtl">
**در پوشه input فایل پیامکی که میخواهید اسپم یا هم(غیر اسپم) بودنش را بفهمید (تست) قرار دهید. **
</div>

نمونه تست غیر اسپم: سلام مواظب خانواده گرامی باشید

In [49]:
from os import listdir
from os.path import isfile, join
import csv
import string
from nltk.corpus import stopwords


def save(filename, dictionary):
    w = csv.writer(open(filename, "w"))
    for key, val in dictionary.items():
        w.writerow([key, val])


def load(filename):
    d = {}
    with open(filename, mode='r') as infile:
        reader = csv.reader(infile)
        for rows in reader:
            if len(rows) == 2:
                d[rows[0]] = float(rows[1])
    return d


def cleaner(text):
    with open('/content/drive/MyDrive/bayes/stop-words.txt', "r") as word_list:
      words = word_list.read().split('\n')
    words.append("\u200c")
    words.append("\u200f")
    digits = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    en = ['C','V','B','N','M','Q','W','E','R','T','Y','U','I','O','P','A','S','D','F','G','H','J','K','L','Z','X','q', 'w','e','r','t','y','u','i','o','p','a','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m']
    for q in digits:
      words.append(str(q))
    for e in en:
      words.append(e)
    words=set(words)
    stopWords = words
    text = text.replace('\n', ' ')
    mail = text.split(' ')
    mail = [word for word in mail if word not in stopWords]
    return mail

def train(filename):
    files = [f for f in listdir(filename) if isfile(join(filename, f))]
    dictionary = {}
    counter = 0
    for file in files:
        file = open(filename + "/" + file, errors="ignore")
        text = file.read()
        file.close()
        mail = cleaner(text)
        for word in mail:
            counter += 1
            if word in dictionary:
                dictionary[word] += 1
            else:
                dictionary[word] = 1
    fileCount = len(files)
    for word in dictionary.keys():
        dictionary[word] = float(dictionary[word]) / fileCount
    dictionary["fileCount"] = fileCount
    return dictionary


def classify(input, diction, priori):
    input = cleaner(input)
    p = 1
    for word in input:
        if word in diction.keys():
            p *= diction[word]
        else:
            p *= 1/diction["fileCount"]+len(diction)+1
    p *= priori
    return p


def trainNsave():
    save("spam_train_results.csv", train("spam"))
    save("ham_train_results.csv", train("ham"))


def test():
    hamDict = load("ham_train_results.csv")
    spamDict = load("spam_train_results.csv")
    files = [f for f in listdir("input") if isfile(join("input", f))]
    spamC = hamC = 0
    for file in files:
        file = open("input" + "/" + file, errors = "ignore")
        text = file.read()
        file.close()
        ham = classify(text, hamDict, 0.5)
        spam = classify(text, spamDict, 0.5)
        if ham <= spam:
            spamC += 1
        else:
            hamC += 1
    print("Spam:\t" + str(hamC))
    print("Ham:\t" + str(spamC))


trainNsave()
test()


Spam:	0
Ham:	1


⚓

--------------------------------------------------------------------------------

In [29]:
with open('/content/drive/MyDrive/bayes/spam-words.txt', "r") as word_list:
  words = word_list.read().split('\n')
spamwords=set(words)

<div dir="rtl">
من این قطعه کد را برای انتقال برخی از پیامک ها از پوشه اولیه پیامکا( که خودم اسمش رو هم یعنی غیر اسپم گذاشتم)، به پوشه اسپم نوشتم. برای اینکار هر پیامکی که تعداد کلمات اسپم ان بر اساس فایل کلمات اسپمی که شما دادید، احتمالش بیشتر از ۰.۶ میشد را میفرستادم تو پوشه اسپم. اینجوری یه دیتاست شامل دو پوشه اسپم و هم دارم که فایلهای تکست داخل انها شامل پیامک های به ترتیب اسپم  و هم هستند.
دلیل این مدلی ایجاد دیتاستم این بود که باید داده هام برچسب گذاری میشدند به تفکیک.
</div>

In [ ]:
#حاصل اجرای این بخش ایجاد پوشه اسپم و هم بود به عنوان دیتاست

import os, shutil
path = "/content/drive/MyDrive/bayes/ham/"
moveto = "/content/drive/MyDrive/bayes/spam/"
files = os.listdir(path)
files.sort()
sp=0
for f in files:
    file = open(path+f, "rt")
    data = file.read()
    w = data.split()
    n=len(w)
    for i in w:
      if i in spamwords:
        sp=sp+1
    p=sp//n
    if p>0.6:
      src = path+f
      dst = moveto+f
      shutil.move(src,dst)